In [ ]:
!pip install datasets transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import time
from tqdm.notebook import trange, tqdm

import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader
from torch import Tensor
from torch.utils.data.dataset import Dataset
import torch.nn.functional as F
from torch.distributions import Categorical
from torch.cuda.amp import autocast, GradScaler

from datasets import load_dataset
from transformers import AutoTokenizer

torch.backends.cuda.matmul.allow_tf32 = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
train_dataset = load_dataset("yahoo_answers_topics", split="train")
test_dataset = load_dataset("yahoo_answers_topics", split="test")

print(f"Length of train dataset: {len(train_dataset)}")
print(f"Length of test dataset: {len(test_dataset)}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.20k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/241M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/270M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/21.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1400000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/60000 [00:00<?, ? examples/s]

Length of train dataset: 1400000
Length of test dataset: 60000


In [ ]:
from bs4 import BeautifulSoup
import html

def clean_text(text):
  try:
    soup = BeautifulSoup(text, "html.parser")
    cleaned = soup.get_text()
    cleaned = html.unescape(cleaned)

    return cleaned
  except Exception as e:
    print("Error cleaning text:", text[:100], "Error:", e)
    return text

In [ ]:
# 5 minutes and 30 seconds
def clean_example(example):
  example["question_title"] = clean_text(example["question_title"])
  example["best_answer"] = clean_text(example["best_answer"])
  return example

train_dataset = train_dataset.map(clean_example)
test_dataset = test_dataset.map(clean_example)

Map:   0%|          | 0/1400000 [00:00<?, ? examples/s]

<ipython-input-4-d44572d35c3c>:6: MarkupResemblesLocatorWarning: The input passed in on this line looks more like a URL than HTML or XML.

If you meant to use Beautiful Soup to parse the web page found at a certain URL, then something has gone wrong. You should use an Python package like 'requests' to fetch the content behind the URL. Once you have the content as a string, you can feed that string into Beautiful Soup.

However, if you want to parse some data that happens to look like a URL, then nothing has gone wrong: you are using Beautiful Soup correctly, and this warning is spurious and can be filtered. To make this warning go away, run this code before calling the BeautifulSoup constructor:

    from bs4 import MarkupResemblesLocatorWarning
    import warnings

    warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)
    
  soup = BeautifulSoup(text, "html.parser")
<ipython-input-4-d44572d35c3c>:6: MarkupResemblesLocatorWarning: The input passed in on this lin

Error cleaning text: <![ %HTML.Frameset; [\n<!-- reserved frame names start with "_" otherwise starts with letter -->\n<! Error: The markup you provided was rejected by the parser. Trying a different parser or a different encoding may help.

Original exception(s) from parser:
 AssertionError: expected name token at '<![ %HTML.Frameset; '


Map:   0%|          | 0/60000 [00:00<?, ? examples/s]

In [ ]:
def remove_slash(example):
  example["question_title"] = example["question_title"].replace("\\n", " ")
  example["best_answer"] = example["best_answer"].replace("\\n", " ")
  return example

train_dataset = train_dataset.map(remove_slash)
test_dataset = test_dataset.map(remove_slash)

Map:   0%|          | 0/1400000 [00:00<?, ? examples/s]

Map:   0%|          | 0/60000 [00:00<?, ? examples/s]

In [ ]:
# train_dataset = train_dataset.select(range(len(train_dataset) // 2))
print(f"Length of train dataset: {len(train_dataset)}")
print(f"Length of test dataset: {len(test_dataset)}")

Length of train dataset: 1400000
Length of test dataset: 60000


In [ ]:
train_dataset[1]["question_title"], train_dataset[1]["best_answer"]

('What is the best off-road motorcycle trail ?',
 'i hear that the mojave road is amazing! search for it online.')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
len(tokenizer.vocab)

30522

In [ ]:
tokenizer.vocab_size

30522

In [ ]:
print("[PAD] token id:", tokenizer.pad_token_id) # 0
print("[CLS] token id:", tokenizer.cls_token_id) # 101
print("[SEP] token id:", tokenizer.sep_token_id) # 102

[PAD] token id: 0
[CLS] token id: 101
[SEP] token id: 102


In [ ]:
class YahooAnswersDataset(Dataset):
    def __init__(self, dataset, tokenizer, max_length=128):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        sample = self.dataset[idx]
        question = sample["question_title"]
        answer = sample["best_answer"]

        encoded_question = self.tokenizer(
            question,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors="pt"
        )

        encoded_answer = self.tokenizer(
            answer,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors="pt"
        )

        # Get rid of the batch dimension
        encoded_question = {key: val.squeeze(dim=0) for key, val in encoded_question.items()}
        encoded_answer = {key: val.squeeze(dim=0) for key, val in encoded_answer.items()}

        return {
            "question": encoded_question,
            "answer": encoded_answer
        }

train_dataset = YahooAnswersDataset(dataset=train_dataset, tokenizer=tokenizer, max_length=32)
test_dataset = YahooAnswersDataset(dataset=test_dataset, tokenizer=tokenizer, max_length=32)

In [ ]:
batch_size = 128

num_workers = os.cpu_count()

train_dataloader = DataLoader(train_dataset,
                              batch_size=batch_size,
                              shuffle=True,
                              num_workers=num_workers,
                              pin_memory=True)
test_dataloader = DataLoader(test_dataset,
                             batch_size=batch_size,
                             shuffle=True,
                             num_workers=num_workers,
                             pin_memory=True)

In [ ]:
test_items = next(iter(train_dataloader))
print(test_items["question"]["input_ids"].shape)
print(test_items["question"]["attention_mask"].shape)
print(test_items["answer"]["input_ids"].shape)
print(test_items["answer"]["attention_mask"].shape)

torch.Size([128, 32])
torch.Size([128, 32])
torch.Size([128, 32])
torch.Size([128, 32])


In [ ]:
print(tokenizer.decode(test_items["question"]["input_ids"][1]))
print(tokenizer.decode(test_items["answer"]["input_ids"][1]))

[CLS] need advice on finding an online feng shui expert? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
[CLS] my master is master peter leung. he lives in toronto, and go to other cities to teach fengshui. you can either contact him at www [SEP]


In [ ]:
class EmbeddingLayer(nn.Module):
  def __init__(self,
               vocab_size: int,
               d_model: int = 512):
    super().__init__()

    self.d_model = d_model

    self.lut = nn.Embedding(num_embeddings=vocab_size, embedding_dim=d_model) # (vocab_size, d_model)

  def forward(self, x):
    # x shape: (batch_size, seq_len)
    return self.lut(x) * math.sqrt(self.d_model) # (batch_size, seq_len, d_model)

In [ ]:
class PositionalEncoding(nn.Module):
  def __init__(self,
               d_model: int = 512,
               dropout: float = 0.1,
               max_length: int = 128):
    super().__init__()

    self.dropout = nn.Dropout(p=dropout)

    pe = torch.zeros(max_length, d_model) # (max_length, d_model)
    # Create position column
    k = torch.arange(0, max_length).unsqueeze(dim=1) # (max_length, 1)

    # Use the log version of the function for positional encodings
    div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)) # (d_model / 2)

    # Use sine for the even indices and cosine for the odd indices
    pe[:, 0::2] = torch.sin(k * div_term)
    pe[:, 1::2] = torch.cos(k * div_term)

    pe = pe.unsqueeze(dim=0) # Add the batch dimension(1, max_length, d_model)

    # We use a buffer because the positional encoding is fixed and not a model paramter that we want to be updated during backpropagation.
    self.register_buffer("pe", pe) # Buffers are saved with the model state and are moved to the correct device

  def forward(self, x):
    # x shape: (batch_size, seq_length, d_model)
    x += self.pe[:, :x.size(1)]
    return self.dropout(x)

In [ ]:
class MultiHeadAttention(nn.Module):
  def __init__(self,
               d_model: int = 512,
               n_heads: int = 8,
               dropout: float = 0.1):
    super().__init__()
    assert d_model % n_heads == 0

    self.d_model = d_model
    self.n_heads = n_heads
    self.d_key = d_model // n_heads

    self.Wq = nn.Linear(in_features=d_model, out_features=d_model)
    self.Wk = nn.Linear(in_features=d_model, out_features=d_model)
    self.Wv = nn.Linear(in_features=d_model, out_features=d_model)
    self.Wo = nn.Linear(in_features=d_model, out_features=d_model)

    self.dropout = nn.Dropout(p=dropout)


  def forward(self,
              query: Tensor,
              key: Tensor,
              value: Tensor,
              mask: Tensor = None):
    # input shape: (batch_size, seq_len, d_model)

    batch_size = key.size(0)

    Q = self.Wq(query)
    K = self.Wk(key)
    V = self.Wv(value)

    Q = Q.view(batch_size, -1, self.n_heads, self.d_key).permute(0, 2, 1, 3) # (batch_size, n_heads, q_length, d_key)
    K = K.view(batch_size, -1, self.n_heads, self.d_key).permute(0, 2, 1, 3) # (batch_size, n_heads, k_length, d_key)
    V = V.view(batch_size, -1, self.n_heads, self.d_key).permute(0, 2, 1, 3) # (batch_size, n_heads, v_length, d_key)

    scaled_dot_product = torch.matmul(Q, K.permute(0, 1, 3, 2)) / math.sqrt(self.d_key) # (batch_size, n_heads, q_length, k_length)

    if mask is not None:
      scaled_dot_product = scaled_dot_product.masked_fill(mask == 0, float('-inf'))

    attention_probs = torch.softmax(scaled_dot_product, dim=-1)

    A = torch.matmul(self.dropout(attention_probs), V)  # (batch_size, n_heads, q_length, d_key)

    A = A.permute(0, 2, 1, 3) # (batch_size, q_length, n_heads, d_key)
    A = A.contiguous().view(batch_size, -1, self.n_heads * self.d_key) # (batch_size, q_length, d_model)

    output = self.Wo(A) # (batch_size, q_length, d_model)

    return output, attention_probs

In [ ]:
class PositionwiseFeedForward(nn.Module):
  def __init__(self,
               d_model: int = 512,
               d_ffn: int = 512,
               dropout: float = 0.1):
    super().__init__()

    self.ffn = nn.Sequential(
        nn.Linear(in_features=d_model, out_features=d_ffn),
        nn.ReLU(),
        nn.Linear(in_features=d_ffn, out_features=d_model),
        nn.Dropout(p=dropout)
    )

  def forward(self, x):
    # x shape: (batch_size, q_length, d_model)
    return self.ffn(x) # (batch_size, q_length, d_model)

In [ ]:
class EncoderLayer(nn.Module):
  def __init__(self,
               d_model: int = 512,
               n_heads: int = 8,
               d_ffn: int = 512,
               dropout: float = 0.1):
    super().__init__()

    self.attention = MultiHeadAttention(d_model=d_model, n_heads=n_heads, dropout=dropout)
    self.attention_layer_norm = nn.LayerNorm(d_model)

    self.position_wise_ffn = PositionwiseFeedForward(d_model=d_model, d_ffn=d_ffn, dropout=dropout)
    self.ffn_layer_norm = nn.LayerNorm(d_model)

    self.dropout = nn.Dropout(p=dropout)

  def forward(self,
              src: Tensor,
              src_mask: Tensor):
    _src, attention_probs = self.attention(query=src, key=src, value=src, mask=src_mask)
    src = self.attention_layer_norm(src + self.dropout(_src))

    _src = self.position_wise_ffn(src)
    src = self.ffn_layer_norm(src + self.dropout(_src))

    return src, attention_probs

In [ ]:
class Encoder(nn.Module):
  def __init__(self,
               d_model: int = 512,
               n_layers: int = 3,
               n_heads: int = 8,
               d_ffn: int = 512,
               dropout: float = 0.1):
    super().__init__()

    self.layers = nn.ModuleList([EncoderLayer(d_model=d_model, n_heads=n_heads, d_ffn=d_ffn, dropout=dropout) for layer in range(n_layers)])
    self.dropout = nn.Dropout(p=dropout)

  def forward(self,
              src: Tensor,
              src_mask: Tensor):

    for layer in self.layers:
      src, attention_probs = layer(src, src_mask)

    self.attention_probs = attention_probs

    # src += torch.randn_like(src) * 0.001
    return src

In [ ]:
class DecoderLayer(nn.Module):
  def __init__(self,
               d_model: int = 512,
               n_heads: int = 8,
               d_ffn: int = 512,
               dropout: float = 0.1):
    super().__init__()

    self.masked_attention = MultiHeadAttention(d_model=d_model, n_heads=n_heads, dropout=dropout)
    self.masked_attention_layer_norm = nn.LayerNorm(d_model)

    self.cross_attention = MultiHeadAttention(d_model=d_model, n_heads=n_heads, dropout=dropout)
    self.cross_attention_layer_norm = nn.LayerNorm(d_model)

    self.position_wise_ffn = PositionwiseFeedForward(d_model=d_model, d_ffn=d_ffn, dropout=dropout)
    self.ffn_layer_norm = nn.LayerNorm(d_model)

    self.dropout = nn.Dropout(p=dropout)

  def forward(self,
              trg: Tensor,
              src: Tensor,
              trg_mask: Tensor,
              src_mask: Tensor):
    _trg, masked_attention_probs = self.masked_attention(query=trg, key=trg, value=trg, mask=trg_mask)
    trg = self.masked_attention_layer_norm(trg + self.dropout(_trg))

    _trg, attention_probs = self.cross_attention(query=trg, key=src, value=src, mask=src_mask)
    trg = self.cross_attention_layer_norm(trg + self.dropout(_trg))

    _trg = self.position_wise_ffn(trg)
    trg = self.ffn_layer_norm(trg + self.dropout(_trg))

    return trg, attention_probs, masked_attention_probs

In [ ]:
class Decoder(nn.Module):
  def __init__(self,
               vocab_size: int,
               d_model: int = 512,
               n_layers: int = 3,
               n_heads: int = 8,
               d_ffn: int = 512,
               dropout: float = 0.1):
    super().__init__()

    self.layers = nn.ModuleList([DecoderLayer(d_model=d_model, n_heads=n_heads, d_ffn=d_ffn, dropout=dropout) for layer in range(n_layers)])
    self.dropout = nn.Dropout(p=dropout)

    self.Wo = nn.Linear(in_features=d_model, out_features=vocab_size)

  def forward(self,
              trg: Tensor,
              src: Tensor,
              trg_mask: Tensor,
              src_mask: Tensor):
    for layer in self.layers:
      trg, attention_probs, masked_attention_probs = layer(trg, src, trg_mask, src_mask)

    self.attention_probs = attention_probs
    self.masked_attention_probs = masked_attention_probs

    # trg += torch.randn_like(trg) * 0.001

    return self.Wo(trg)

In [ ]:
class Transformer(nn.Module):
  def __init__(self,
               encoder: Encoder,
               decoder: Decoder,
               src_embed: EmbeddingLayer,
               trg_embed: EmbeddingLayer,
               src_pad_idx: int,
               trg_pad_idx: int,
               device):
    super().__init__()

    self.encoder = encoder
    self.decoder = decoder
    self.src_embed = src_embed
    self.trg_embed = trg_embed
    self.device = device
    self.src_pad_idx = src_pad_idx
    self.trg_pad_idx = trg_pad_idx

  def make_src_mask(self, src: Tensor):
    # Assign 1 to tokens that need attended to and 0 to padding tokens, then add 2 dimensions
    src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)

    return src_mask

  def make_trg_mask(self, trg: Tensor):
    seq_length = trg.shape[1]

    # Assign True to tokens that need attended to and False to padding tokens, then add 2 dimensions
    trg_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2) # (batch_size, 1, 1, seq_length)

    # Generate subsequent mask
    trg_sub_mask = torch.tril(torch.ones((seq_length, seq_length), device=self.device)).bool() # (batch_size, 1, seq_length, seq_length)

    # Bottom triangle is True, top triangle is False
    trg_mask = trg_mask & trg_sub_mask

    return trg_mask

  def forward(self, src: Tensor, trg: Tensor):
    src_mask = self.make_src_mask(src) # (batch_size, 1, 1, src_seq_length)

    trg_mask = self.make_trg_mask(trg) # (batch_size, 1, trg_seq_length, trg_seq_length)

    src = self.encoder(self.src_embed(src), src_mask)  # (batch_size, src_seq_length, d_model)
    output = self.decoder(self.trg_embed(trg), src, trg_mask, src_mask)

    return output

In [ ]:
def make_model(device,
               tokenizer,
               n_layers: int = 3,
               d_model: int = 512,
               d_ffn: int = 2048,
               n_heads: int = 8,
               dropout: float = 0.1,
               max_length: int = 128):
  encoder = Encoder(d_model=d_model,
                    n_layers=n_layers,
                    n_heads=n_heads,
                    d_ffn=d_ffn,
                    dropout=dropout)

  decoder = Decoder(vocab_size=tokenizer.vocab_size,
                    d_model=d_model,
                    n_layers=n_layers,
                    n_heads=n_heads,
                    d_ffn=d_ffn,
                    dropout=dropout)

  src_embed = EmbeddingLayer(vocab_size=tokenizer.vocab_size, d_model=d_model)
  trg_embed = EmbeddingLayer(vocab_size=tokenizer.vocab_size, d_model=d_model)

  pos_enc = PositionalEncoding(d_model=d_model,
                               dropout=dropout,
                               max_length=max_length)

  model = Transformer(encoder=encoder,
                      decoder=decoder,
                      src_embed=nn.Sequential(src_embed, pos_enc),
                      trg_embed=nn.Sequential(trg_embed, pos_enc),
                      src_pad_idx=tokenizer.pad_token_id,
                      trg_pad_idx=tokenizer.pad_token_id,
                      device=device)

  # Initialize parameters with Xaviar/Glorot
  # This maintains a consistent variance of activations throughout the network
  # Helps avoid issues like vanishing or exploding gradients.
  for p in model.parameters():
    if p.dim() > 1:
      nn.init.xavier_uniform_(p)

  return model

In [ ]:
model = make_model(device=device,
                   tokenizer=tokenizer,
                   n_layers=4,
                   d_model=512,
                   d_ffn=2048,
                   n_heads=8,
                   dropout=0.1,
                   max_length=32)
# model = make_model(device=device,
#                    tokenizer=tokenizer,
#                    n_layers=3,
#                    d_model=192,
#                    d_ffn=192,
#                    n_heads=8,
#                    dropout=0.1,
#                    max_length=32)
model.to(device)

Transformer(
  (encoder): Encoder(
    (layers): ModuleList(
      (0-3): 4 x EncoderLayer(
        (attention): MultiHeadAttention(
          (Wq): Linear(in_features=512, out_features=512, bias=True)
          (Wk): Linear(in_features=512, out_features=512, bias=True)
          (Wv): Linear(in_features=512, out_features=512, bias=True)
          (Wo): Linear(in_features=512, out_features=512, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (attention_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (position_wise_ffn): PositionwiseFeedForward(
          (ffn): Sequential(
            (0): Linear(in_features=512, out_features=2048, bias=True)
            (1): ReLU()
            (2): Linear(in_features=2048, out_features=512, bias=True)
            (3): Dropout(p=0.1, inplace=False)
          )
        )
        (ffn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=Fals

In [ ]:
print(f'The model has {(sum(p.numel() for p in model.parameters() if p.requires_grad)):,} trainable parameters')

The model has 76,337,978 trainable parameters


In [ ]:
lr = 1e-4

optimizer = torch.optim.Adam(params=model.parameters(),
                             lr=lr,
                             betas=(0.5, 0.999))
loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id) # In loss calculation, ignore any target tokens that have an index of PAD_IDX
scaler = GradScaler()

<ipython-input-29-2462064e2f33>:7: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/usr/local/lib/python3.11/dist-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


In [ ]:
def train(model,
          iterator,
          optimizer,
          loss_fn,
          clip,
          epoch):
  model.train()
  epoch_loss = 0

  pbar = tqdm(iterator, total=len(iterator), desc=f"Epoch {epoch+1} Progress", colour="#005500")
  for i, batch in enumerate(pbar):
    src = batch["question"]["input_ids"].to(device)
    trg = batch["answer"]["input_ids"].to(device)

    optimizer.zero_grad()
    with autocast():
      # Forward pass
      # Pass in the full target sequence without the <eos> token, and in the model we use the target mask to prevent it from peeking at future values
      # We also use this for teacher forcing with the target mask when the target sequence is put over the target mask to get a matrix that represents the different sequences that we use for the trgs. Each new row of the matrix is a sequence with one more word for teacher forcing.
      logits = model(src, trg[:,:-1]) # Remove <eos> token because we want to predict it ourselves

      # Output shape: (batch_size, trg_seq_length - 1, vocab_size) trg_seq_length - 1 is made up of the different teacher-forced sequences

      # Expected target
      expected_output = trg[:,1:] # Remove <bos> token because the <eos> was not generated in the logits, so we need to remove it to properly compare

      # Calculate the loss
      # contiguous() flattens so that every token position in every sequence is treated as an individual prediction.
      loss = loss_fn(logits.contiguous().view(-1, logits.shape[-1]),
                      expected_output.contiguous().view(-1))

    scaler.scale(loss).backward()
    scaler.unscale_(optimizer)
    torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
    scaler.step(optimizer)
    scaler.update()
    epoch_loss += loss.item()

    pbar.set_postfix(loss=loss.item()) # Update the loss on the tqdm progress bar

  return (epoch_loss / len(iterator))

In [ ]:
def evaluate(model,
             iterator,
             loss_fn):
  model.eval()
  epoch_loss = 0

  with torch.inference_mode():
    for i, batch in enumerate(iterator):
      src = batch["question"]["input_ids"].to(device)
      src_attention_mask = batch["question"]["attention_mask"].to(device)

      trg = batch["answer"]["input_ids"].to(device)
      trg_attention_mask = batch["answer"]["attention_mask"].to(device)

      # Forward pass
      # Pass in the full target sequence without the <eos> token, and in the model we use the target mask to prevent it from peeking at future values
      logits = model(src, trg[:,:-1]) # Remove <eos> token because we want to predict it ourselves

      # expected target
      expected_output = trg[:,1:] # Remove <bos> token because the <eos> was not generated in the logits, so we need to remove it to properly compare

      # Calculate the loss
      # contiguous() flattens so that every token position in every sequence is treated as an individual prediction.
      loss = loss_fn(logits.contiguous().view(-1, logits.shape[-1]),
                      expected_output.contiguous().view(-1))
      epoch_loss += loss.item()

  return (epoch_loss / len(iterator))

In [ ]:
def epoch_time(start_time, end_time):
  elapsed_time = end_time - start_time
  elapsed_mins = int(elapsed_time / 60)
  elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
  return elapsed_mins, elapsed_secs

In [ ]:
epochs = 10
clip = 1

best_valid_loss = float('inf')
model_path = "question_answering_model.pt"

if os.path.exists(model_path):
  print(f"Loading model from {model_path}...")
  model.load_state_dict(torch.load(model_path, map_location=device))

Loading model from question_answering_model.pt...


In [ ]:
should_train = True

if should_train:
  for epoch in tqdm(range(epochs), desc=f"Training progress", colour="#00ff00"):
    start_time = time.time()

    train_loss = train(model=model,
                      iterator=train_dataloader,
                      optimizer=optimizer,
                      loss_fn=loss_fn,
                      clip=clip,
                      epoch=epoch)

    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    message = f"Epoch: {epoch + 1} | Time: {epoch_mins}m {epoch_secs}s --> STORED"

    torch.save(model.state_dict(), model_path)

    print(message)
    print(f"Train Loss: {train_loss:.3f} | Train Perplexity: {math.exp(train_loss):7.3f}")

Training progress:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1 Progress:   0%|          | 0/10938 [00:00<?, ?it/s]

<ipython-input-30-11b492cce893>:16: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/usr/local/lib/python3.11/dist-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
test_loss = evaluate(model=model,
                     iterator=test_dataloader,
                     loss_fn=loss_fn)

print(f'Test Loss: {test_loss:.3f} | Test Perplexity: {math.exp(test_loss):7.3f}')

KeyboardInterrupt: 

In [ ]:
tokenizer.decode(tokenizer.encode("What is the weather today?"))

'[CLS] what is the weather today? [SEP]'

In [ ]:
def answer_question(question, model, device, max_length: int = 32):
  model.eval()

  src = tokenizer.encode(question)

  src_tensor = torch.tensor(src).unsqueeze(0).to(device)

  trg_indexes = [tokenizer.cls_token_id]

  for i in range(max_length):
    trg_tensor = torch.tensor(trg_indexes).unsqueeze(0).to(device)

    # Generate the next token
    with torch.inference_mode():
      logits = model.forward(src_tensor, trg_tensor) # Generate logits for next word using the current target sequence

      # Logits shape: (batch_size=1, seq_length, vocab_size)

      temp = 0.35

      # Randomly sample based off of the probabilities (highest prob has highest chance of being chosen), and get the predicted token from the vocab_size
      dist = Categorical(logits=logits[:, -1] / temp) # Take the logits for the last token (newly generated token) from the output; Shape: (1, vocab_size)
      pred_token = dist.sample().reshape(1, 1).item()

      # If <eos> token or max length, stop generating
      if pred_token == tokenizer.sep_token_id or i == (max_length - 1):
        # Output
        trg_output = tokenizer.decode(trg_indexes)

        return src, trg_output, model.decoder.attention_probs, model.decoder.masked_attention_probs
      else:
        # Add the token to the indices for the next prediction
        trg_indexes.append(pred_token)

In [ ]:
src_sentence = "Why did the chicken cross the road?"

src, trg_output, attention_probs, masked_attention_probs = answer_question(src_sentence, model, device)

print(f"target output = {trg_output}")

target output = [CLS] because he was a chicken.


In [ ]:
#@title Question Answering
src_sentence = "Should I buy a house?" #@param ""

src, trg_output, attention_probs, masked_attention_probs = answer_question(src_sentence, model, device)

print(f"target output = {trg_output}")

target output = [CLS] i think the best way torange your question is to find out how much you can afford. if you want to buy a house, go for
